## Sentiment Analysis
Now, let's see a recurrent neural network in action through an example called "sentiment analysis."

Sentiment analysis is about using computers to figure out opinions in text. Specifically, it's about determining if a piece of writing, like a movie review, expresses a positive, negative, or neutral feeling toward something.

For this example, we'll focus on classifying movie reviews into positive, negative, or neutral categories. It's a practical application of recurrent neural networks.

### Movie Review Dataset
Well start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
!pip install Keras-Preprocessing
from keras_preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00


In [ ]:
train_data[1] # Lets look at one review

### More Preprocessing
If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore, we must make each review the same length. To do this we will follow the procedure below:
- if the review is greater than 250 words then trim off the extra words
- if the review is less than 250 words add the necessary amount of 0's to make it equal to 250.

Luckily for us keras has a function that can do this for us:




In [4]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

### Creating the Model
Now it's time to create the model. We'll use a **word embedding layer** as the first layer in our model and add a **LSTM layer** afterwards that feeds into a dense node to get our predicted sentiment.

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


### Training
Now it's time to compile and train the model.

In [7]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 71s 102ms/step - loss: 0.4738 - acc: 0.7699 - val_loss: 0.3339 - val_acc: 0.8594
Epoch 2/10
625/625 [==============================] - 24s 39ms/step - loss: 0.2667 - acc: 0.8963 - val_loss: 0.3148 - val_acc: 0.8656
Epoch 3/10
625/625 [==============================] - 14s 22ms/step - loss: 0.2038 - acc: 0.9240 - val_loss: 0.3308 - val_acc: 0.8726
Epoch 4/10
625/625 [==============================] - 12s 19ms/step - loss: 0.1648 - acc: 0.9401 - val_loss: 0.3374 - val_acc: 0.8736
Epoch 5/10
625/625 [==============================] - 10s 16ms/step - loss: 0.1379 - acc: 0.9516 - val_loss: 0.3217 - val_acc: 0.8836
Epoch 6/10
625/625 [==============================] - 8s 13ms/step - loss: 0.1149 - acc: 0.9610 - val_loss: 0.3167 - val_acc: 0.8830
Epoch 7/10
625/625 [==============================] - 10s 16ms/step - loss: 0.0959 - acc: 0.9693 - val_loss: 0.3902 - val_acc: 0.8802
Epoch 8/10
625/625 [==============================] - 9s 14ms/

And we'll evaluate the model on our training data to see how well it performs.

In [8]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 4s 5ms/step - loss: 0.5716 - acc: 0.8528
[0.5715908408164978, 0.852840006351471]


So we're scoring somewhere in the mid-high 80's. Not bad for a simple recurrent network.

###Making Predictions
Now let’s use our network to make predictions on our own reviews.

Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well load the encodings from the dataset and use them to encode our own data.




In [9]:
# Get the word index for the IMDb dataset
word_index = imdb.get_word_index()

def encode_text(text):
  # Convert the input text into a sequence of tokens
  tokens = keras.preprocessing.text.text_to_word_sequence(text)

  # Map tokens to their corresponding word index values (or 0 if not found)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


1641221/1641221 [==============================] - 1s 1us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [10]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [11]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


1/1 [==============================] - 1s 674ms/step
[0.9660795]
1/1 [==============================] - 0s 35ms/step
[0.27064502]


The prediction results are as follows:

- For the positive review: [0.9660795]
- For the negative review: [0.27064502]

Higher values indicate **"positive review"** and Lower values indicate **"negative review"** in the predicted sentiment.